In [9]:
import numpy as np
import torch
import random

In [10]:
number_of_nodes = 5
dimensions = 128
walk_length = 80
num_walks = 10
p = 1
q = 1

In [17]:
A = np.random.randint(0, 2, size=(number_of_nodes, number_of_nodes))
A = (((A + A.T) / 2) > 0) * 1.
np.fill_diagonal(A, 0.)
graph = torch.from_numpy(A).numpy()

In [18]:
first_walk_probability = np.zeros(graph.shape)  
walk_probability = np.zeros((len(graph), len(graph), len(graph)))

In [19]:
print("Computing prob for first walk...")
sum_weight = np.sum(graph, axis=1)
sum_weight = np.where(sum_weight == 0, 1, sum_weight)
normalized_weight = graph / np.tile(sum_weight, (len(graph), 1)).T
first_walk_probability = normalized_weight

# Afterwards
print("Computing prob for second walk...")
for last in range(len(graph)):
    for current in range(len(graph)):
        if graph[last, current] == 0:
            continue
        unbiased_unnormalized_weight = graph[current, :]
        for destination in range(len(graph)):
            if destination == last:
                unbiased_unnormalized_weight[last] = unbiased_unnormalized_weight[last] / p
            elif graph[last, destination] == 0:
                unbiased_unnormalized_weight[last] = unbiased_unnormalized_weight[last] / q
        biased_normalized_weight = unbiased_unnormalized_weight / np.sum(unbiased_unnormalized_weight)
        walk_probability[last, current, :] = biased_normalized_weight

Computing prob for first walk...
Computing prob for second walk...


In [20]:
all_walks = []

for n_walk in range(num_walks):

    n_th_walks = []

    for source in range(len(graph)):

        walk = [source]
     

        if np.sum(first_walk_probability[source, :]) != 0:

            first_dest = np.random.choice(np.arange(len(graph)), 1, p=first_walk_probability[source, :])[0]
            walk.append(first_dest)
        
        else:

            walk = [str(source)]
            n_th_walks.append(walk)
            continue

        while len(walk) < walk_length:

            if np.sum(walk_probability[walk[-2], walk[-1], :]) != 0:

                dest = np.random.choice(np.arange(len(graph)), 1, p=walk_probability[walk[-2], walk[-1], :])[0]
                walk.append(dest)

            else:
                break

        walk = [str(elm) for elm in walk]
        n_th_walks.append(walk)

    random.shuffle(n_th_walks)
    all_walks += n_th_walks

In [24]:
np.array(all_walks).shape

(50, 80)